In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import BertTokenizerFast, BertModel
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
import string
import json

In [48]:
lang = 'telugu'

# Load train data from json
with open(f'/kaggle/input/llm-project/{lang}_train.json') as f:
    train_data = json.load(f)

# Load test data from json
with open(f'/kaggle/input/llm-project/{lang}_test.json') as f:
    test_data = json.load(f)

In [49]:
# Load tokeniser for MBERT
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [50]:
class NER_Dataset(Dataset):
    def __init__(self, data, tokenizer, max_len=128):
        self.data = data  # List of sentences and labels
        self.tokenizer = tokenizer  # mBERT tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]["tokens"]
        word_labels = self.data[idx]["ner_tags"]

        # Tokenize the text and align the labels
        encoding = self.tokenizer(text,
                                  is_split_into_words=True,
                                  return_offsets_mapping=True,
                                  padding='max_length',
                                  truncation=True,
                                  max_length=self.max_len)

        labels = [self.align_labels(encoding['offset_mapping'], word_labels)]

        # Remove the offset mapping to prevent issues during model training
        del encoding['offset_mapping']

        item = {key: torch.tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.tensor(labels[0], dtype=torch.long)

        return item
    
    # Create a function to align labels
    def align_labels(self, offset_mapping, labels):
        aligned_labels = []
        current_label_index = 0

        for offset in offset_mapping:
            # If the offset mapping is (0, 0), it's a special token ([CLS], [SEP], [PAD])
            if offset == (0, 0):
                aligned_labels.append(-100)  # -100 is used to ignore these tokens in the loss computation
            else:
                # Check if the token is the start of a new word
                if offset[0] == 0:
                    aligned_labels.append(labels[current_label_index])
                    current_label_index += 1
                else:
                    # If the token is not the first subtoken, you can decide how to label it. 
                    # For simplicity, let's use the same label as the first subtoken
                    aligned_labels.append(labels[current_label_index - 1])

        return aligned_labels

In [51]:
# Create train dataset and test dataset
train_dataset = NER_Dataset(train_data, tokenizer)
test_dataset = NER_Dataset(test_data, tokenizer)

# Create train dataloader and test dataloader
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [52]:
from transformers import AutoTokenizer, AutoModel

# Model for NER
class MBERT_NER(nn.Module):
    def __init__(self, num_labels, gru_hidden_size, num_gru_layers, freeze_bert=False):
        super(MBERT_NER, self).__init__()
        self.bert = AutoModel.from_pretrained("ai4bharat/indic-bert")
        self.gru = nn.GRU(input_size=self.bert.config.hidden_size,
                          hidden_size=gru_hidden_size,
                          num_layers=num_gru_layers,
                          batch_first=True)
        self.classifier = nn.Linear(gru_hidden_size, num_labels)
        self.dropout = nn.Dropout(0.1)
        self.batch_norm = nn.BatchNorm1d(gru_hidden_size)

        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        gru_output, _ = self.gru(sequence_output)
        gru_output = self.batch_norm(gru_output)
        gru_output = self.dropout(gru_output)
        logits = self.classifier(gru_output)
        return logits

# Create the NER model
NUM_LABELS = 7 # Number of NER tags
GRU_HIDDEN_SIZE = 128 # Hidden size of the GRU
NUM_GRU_LAYERS = 1 # Number of layers in the GRU
FREEZE_BERT = False # Whether to freeze the BERT model
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

model = MBERT_NER(num_labels=NUM_LABELS,
                    gru_hidden_size=GRU_HIDDEN_SIZE,
                    num_gru_layers=NUM_GRU_LAYERS,
                    freeze_bert=FREEZE_BERT)

model.to(DEVICE)

MBERT_NER(
  (bert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(200000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertSdpaAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
       

In [53]:
from sklearn.metrics import f1_score, accuracy_score

# Optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss(ignore_index=-100)

# Training loop
EPOCHS = 6

train_losses = []
train_f1_scores = []
train_acc_scores = []
val_losses = []
val_f1_scores = []
val_acc_scores = []


for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    train_predictions, train_labels = [], []

    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        logits = logits.view(-1, NUM_LABELS)
        labels = labels.view(-1)

        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # Get predictions and filter out ignored indices for metric calculations
        predictions = torch.argmax(logits, dim=-1)
        active_indices = labels != -100
        train_predictions.extend(predictions[active_indices].cpu().numpy())
        train_labels.extend(labels[active_indices].cpu().numpy())

    train_loss /= len(train_dataloader)
    train_losses.append(train_loss)

    train_f1 = f1_score(train_labels, train_predictions, average='macro')
    train_f1_scores.append(train_f1)

    train_acc = accuracy_score(train_labels, train_predictions)
    train_acc_scores.append(train_acc)

    # Validation loop
    model.eval()
    val_loss = 0
    val_predictions, val_labels = [], []

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)

            logits = model(input_ids, attention_mask)
            logits = logits.view(-1, NUM_LABELS)
            labels = labels.view(-1)

            loss = criterion(logits, labels)
            val_loss += loss.item()

            # Filter predictions and labels
            predictions = torch.argmax(logits, dim=-1)
            active_indices = labels != -100
            val_predictions.extend(predictions[active_indices].cpu().numpy())
            val_labels.extend(labels[active_indices].cpu().numpy())

    val_loss /= len(test_dataloader)
    val_losses.append(val_loss)

    val_f1 = f1_score(val_labels, val_predictions, average='macro')
    val_f1_scores.append(val_f1)

    val_acc = accuracy_score(val_labels, val_predictions)
    val_acc_scores.append(val_acc)

    print(f"Epoch {epoch + 1}/{EPOCHS}, Train Loss: {train_loss:.4f}, Train F1: {train_f1:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val F1: {val_f1:.4f}, Val Acc: {val_acc:.4f}")

Epoch 1/6, Train Loss: 1.1855, Train F1: 0.1365, Train Acc: 0.6479, Val Loss: 1.0643, Val F1: 0.1945, Val Acc: 0.6843
Epoch 2/6, Train Loss: 1.0081, Train F1: 0.2419, Train Acc: 0.6955, Val Loss: 0.9476, Val F1: 0.2820, Val Acc: 0.7184
Epoch 3/6, Train Loss: 0.9012, Train F1: 0.3379, Train Acc: 0.7253, Val Loss: 0.9365, Val F1: 0.3710, Val Acc: 0.7148
Epoch 4/6, Train Loss: 0.8200, Train F1: 0.4197, Train Acc: 0.7502, Val Loss: 0.8255, Val F1: 0.3745, Val Acc: 0.7448
Epoch 5/6, Train Loss: 0.7230, Train F1: 0.4982, Train Acc: 0.7785, Val Loss: 0.7982, Val F1: 0.4550, Val Acc: 0.7606
Epoch 6/6, Train Loss: 0.6423, Train F1: 0.5554, Train Acc: 0.8006, Val Loss: 0.7590, Val F1: 0.5289, Val Acc: 0.7713


In [54]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Load the test data from json for all 5 languages
languages = ['hindi', 'bengali', 'marathi', 'tamil', 'telugu']

# Iterate over all languages and evaluate the model
for lang in languages:
    with open(f'/kaggle/input/llm-project/{lang}_test.json') as f:
        test_data = json.load(f)

    test_dataset = NER_Dataset(test_data, tokenizer)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    model.eval()
    test_predictions, test_labels = [], []

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)

            logits = model(input_ids, attention_mask)
            logits = logits.view(-1, NUM_LABELS)
            labels = labels.view(-1)

            # Filter predictions and labels
            predictions = torch.argmax(logits, dim=-1)
            active_indices = labels != -100
            test_predictions.extend(predictions[active_indices].cpu().numpy())
            test_labels.extend(labels[active_indices].cpu().numpy())

    weighted_f1 = f1_score(test_labels, test_predictions, average='weighted')
    macro_f1 = f1_score(test_labels, test_predictions, average='macro')
    accuracy = accuracy_score(test_labels, test_predictions)

    if lang == 'hindi':
        LANG = "Hindi"
    elif lang == 'bengali':
        LANG = "Bengali"
    elif lang == 'marathi':
        LANG = "Marathi"
    elif lang == 'tamil':
        LANG = "Tamil"
    elif lang == 'telugu':
        LANG = "Telugu"

    print(f"Language: {LANG}")
    print()
    print(f"Weighted F1: {weighted_f1:.4f}")
    print(f"Macro F1: {macro_f1:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print()
    print("CLASSIFICATION REPORT: ")
    print(classification_report(test_labels, test_predictions))
    print()
    print("CONFUSION MATRIX: ")
    print(confusion_matrix(test_labels, test_predictions))
    print("--------------------------------------------------")

Language: Hindi

Weighted F1: 0.3275
Macro F1: 0.2196
Accuracy: 0.3673

CLASSIFICATION REPORT: 
              precision    recall  f1-score   support

           0       0.49      0.71      0.58      5277
           1       0.11      0.04      0.06      1481
           2       0.16      0.10      0.13      1874
           3       0.15      0.25      0.18       986
           4       0.34      0.26      0.29      2602
           5       0.26      0.27      0.27      1195
           6       0.18      0.01      0.03       867

    accuracy                           0.37     14282
   macro avg       0.24      0.24      0.22     14282
weighted avg       0.32      0.37      0.33     14282


CONFUSION MATRIX: 
[[3735  112  240  632  367  171   20]
 [ 706   55   49  273  177  219    2]
 [ 852   59  196  144  450  165    8]
 [ 427   99   34  249   88   85    4]
 [ 961   85  473  212  672  184   15]
 [ 548   83   54  124   52  327    7]
 [ 346   26  151   75  173   84   12]]
--------------------

In [46]:
# Save the model
torch.save(model.state_dict(), f'NER_FineTune_telugu.pth')